This notebook is a modified version of the original code made by TensorFlow 

https://www.tensorflow.org/tutorials/sequences/text_generation

In [5]:
!wget https://raw.githubusercontent.com/SaifSultanAlharthi/Arabic_Poetry_Deep_Learning/main/poems.txt

'wget' is not recognized as an internal or external command,
operable program or batch file.


### Import TensorFlow and other libraries

In [6]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.executing_eagerly()

import numpy as np
import os
import time
import glob

### Read the data

First, look in the text.

In [10]:
# Read, then decode for py2 compat.
text = open('poems.txt', 'rb').read().decode(encoding='utf-8')
  
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

# remove some exteranous chars 
execluded = '!()*-.1:=[]«»;؛,،~?؟#\u200f\ufeff'
out = ""


for char in text:
  if char not in execluded:
    out += char
text = out
text = text.replace("\t\t\t", "\t")
text = text.replace("\r\r\n", "\n")
text = text.replace("\r\n","\n")
text = text.replace("\t\n", "\n")
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

Length of text: 293245 characters
43 unique characters


In [11]:
# Take a look at the first 250 characters in text
print(text[:250])

يابير طيك طي واعماقك اعماق

ويعيش بك داب وحمامه وعصفور

فاضت عيونك ماك وابتلت احداق

وتنهدت منك محاله الناعور

يوم انبطت روحك سجدتي لخلاق

وعقب ارتفع جمك ضحكتي مع النور

ويوم انهمر دمعك خضعتي على ساق

ويوم انكسر حجلك زعلتي على الحور

ياما بك الرمان ز


## Process the text

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [12]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to `len(unique)`.

In [13]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '_' :   2,
  'i' :   3,
  'ء' :   4,
  'آ' :   5,
  'أ' :   6,
  'ؤ' :   7,
  'إ' :   8,
  'ئ' :   9,
  'ا' :  10,
  'ب' :  11,
  'ت' :  12,
  'ث' :  13,
  'ج' :  14,
  'ح' :  15,
  'خ' :  16,
  'د' :  17,
  'ذ' :  18,
  'ر' :  19,
  ...
}


In [14]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'يابير طيك طي ' ---- characters mapped to int ---- > [39 10 11 39 19  1 25 39 32  1 25 39  1]


### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [15]:
# The maximum length sentence we want for a single input in characters
seq_length = 200
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

ي
ا
ب
ي
ر


The `batch` method lets us easily convert these individual characters to sequences of the desired size.

In [16]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'يابير طيك طي واعماقك اعماق\n\nويعيش بك داب وحمامه وعصفور\n\nفاضت عيونك ماك وابتلت احداق\n\nوتنهدت منك محاله الناعور\n\nيوم انبطت روحك سجدتي لخلاق\n\nوعقب ارتفع جمك ضحكتي مع النور\n\nويوم انهمر دمعك خضعتي على ساق\n\n'
'ويوم انكسر حجلك زعلتي على الحور\n\nياما بك الرمان زهر ف الاوراق\n\nلين اكتنز في حلق نحله ودبور\n\nوياكم وردك وصدرك ضين ونياق\n\nوياكم بهج جمك بعارين وصدور\n\nوياما رطب نخلك تعسل ف الارياق\n\nوياما طحين ارضك تقسم ع'
'لى الدور\n\nوياما وهبتي جل وافنيتي ادقاق\n\nوياما هماجك حل في الخصب والبور\n\nشفتك وانا قلبي على صلدك ارقاق\n\nوحسيت في حنجرتي عظام وكسور\n\nيقولون الشعر وصل القلم والصفحه الطرقى\n\nوانا اقول الشعر قدح الخيال ونار'
' صوانه\n\nحبيبي والبحر سور التراب ودفتر الغرقى\n\nلو اني ماكتبتك لامني ملحه ومرجانه\n\nحبيبي لاهمت حبر العروق الغيمه الزرقا\n\nعلى رمل الورق دمي سحاب هل هتانه\n\nحبيبي كل ماذعذع جفاك وناحت الورقى\n\nيتل البال قلبي'
' واهدي السجات جنحانه\n\nحبيبي وانت مرقاب الشجون وهاجس المرقى\n\nترى كلٍ يموت الا الوفي بعيون خلانه\n\nحبيبي والجروح اللي سببها ليله ا

For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [17]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [18]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'يابير طيك طي واعماقك اعماق\n\nويعيش بك داب وحمامه وعصفور\n\nفاضت عيونك ماك وابتلت احداق\n\nوتنهدت منك محاله الناعور\n\nيوم انبطت روحك سجدتي لخلاق\n\nوعقب ارتفع جمك ضحكتي مع النور\n\nويوم انهمر دمعك خضعتي على ساق\n'
Target data: 'ابير طيك طي واعماقك اعماق\n\nويعيش بك داب وحمامه وعصفور\n\nفاضت عيونك ماك وابتلت احداق\n\nوتنهدت منك محاله الناعور\n\nيوم انبطت روحك سجدتي لخلاق\n\nوعقب ارتفع جمك ضحكتي مع النور\n\nويوم انهمر دمعك خضعتي على ساق\n\n'


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and tries to predict the index for "i" as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [19]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 39 ('ي')
  expected output: 10 ('ا')
Step    1
  input: 10 ('ا')
  expected output: 11 ('ب')
Step    2
  input: 11 ('ب')
  expected output: 39 ('ي')
Step    3
  input: 39 ('ي')
  expected output: 19 ('ر')
Step    4
  input: 19 ('ر')
  expected output: 1 (' ')


### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [20]:
# Batch size
BATCH_SIZE = 128
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((128, 200), (128, 200)), types: (tf.int32, tf.int32)>

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [21]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

Next define a function to build the model.

In [22]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
        return_sequences=True,
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [23]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-liklihood of the next character:

![A drawing of the data passing through the model](https://tensorflow.org/tutorials/sequences/images/text_generation_training.png)

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [24]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(128, 200, 43) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (128, None, 256)          11008     
                                                                 
 gru (GRU)                   (128, None, 1024)         3938304   
                                                                 
 dense (Dense)               (128, None, 43)           44075     
                                                                 
Total params: 3,993,387
Trainable params: 3,993,387
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [26]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [27]:
print(len(sampled_indices))
sampled_indices

200


array([42,  2, 23, 36, 12, 42, 31, 28, 20, 30, 36, 42, 42, 30,  0, 36, 33,
       17, 29, 22, 16, 31, 40, 21, 32, 28, 40, 34, 33, 38,  8, 39, 28, 22,
       32, 37, 28, 20, 33, 28, 15, 42, 33, 21, 39,  3, 23, 12, 15, 19, 18,
       32, 27, 40,  1,  3, 28, 32, 26, 11,  1,  0,  8, 16, 17, 14,  8, 34,
        6,  9, 10, 12,  6, 26, 42, 10, 18, 32,  0,  6,  3, 17, 22,  8, 18,
       20, 27, 13, 22, 23, 16, 24, 31, 13, 19, 42, 28,  1, 39, 23, 17, 18,
        2,  2, 13, 28, 22, 15, 16, 40, 26, 37, 37, 35, 14,  2, 31, 34, 11,
       31,  6, 30, 32, 34, 20, 39, 31, 13,  2, 30, 20, 42, 37, 29, 13, 37,
       15, 22, 36,  3, 37, 32, 40, 25, 42,  8, 18,  2,  3, 25,  5, 19, 31,
       16, 29, 17, 17, 35, 27, 34, 13, 39, 30, 34,  8, 18, 26, 14, 37, 34,
       38,  1, 27, 32,  4, 37, 23,  9,  4, 37, 32, 40, 23,  8, 32, 39, 26,
        6, 18,  1, 12,  6,  0, 34, 33, 17,  8, 38, 42,  4], dtype=int64)

Decode these to see the text predicted by this untrained model:

In [28]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0].numpy()])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ـيـهْ\nأذئـاب أقـفـارٍ إذا مــا لــم يـكـنْحربا وإن حمِي الوطيـس حصانيـه\nأولـم تكـن تــدري سـنـاء بأنـنـيلا يشتكـي ضـرب الـرقـابِ سِنانـيـهْ\nأولـم تكـن تــدري سـنـاء بأنـنـيأعْـطـي إذا رب الـمـلا أعطـا'

Next Char Predictions: 
 'ْ_صهتْقغزفهْْف\nهلدـشخقٍسكغٍملىإيغشكوغزلغحْلسيiصتحرذكعٍ iغكظب \nإخدجإمأئاتأظْاذك\nأiدشإذزعثشصخضقثرْغ يصدذ__ثغشحخٍظوونج_قمبقأفكمزيقث_فزْوـثوحشهiوكٍطْإذ_iطآرقخـددنعمثيفمإذظجومى عكءوصئءوكٍصإكيظأذ تأ\nملدإىْء'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_softmax_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.


In [29]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (128, 200, 43)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       3.7610064


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.train.AdamOptimizer` with default arguments and the loss function.

In [32]:
model.compile(
    optimizer = tf.optimizers.Adam(),
    loss = loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [33]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 3 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [34]:
EPOCHS=50

In [ ]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch,
                    callbacks=[checkpoint_callback])

Epoch 1/50
 5/11 [============>.................] - ETA: 1:00 - loss: 4.6600

## Generate text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            10240     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3935232   
_________________________________________________________________
dense_1 (Dense)              (1, None, 40)             41000     
Total params: 3,986,472
Trainable params: 3,986,472
Non-trainable params: 0
_________________________________________________________________


### The prediction loop

The following code block generates the text:

* It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Then, use a multinomial distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one word. After predicting the next word, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted words.


![To generate text the model's output is fed back to the input](https://tensorflow.org/tutorials/sequences/images/text_generation_sampling.png)

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      #print(tf.multinomial(predictions, num_samples=1).shape)
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"سلام"))

Instructions for updating:
Use tf.random.categorical instead.
سلام وعزمة
وعيني بأنواع المله وإنها
نتى رشد محبورة اخبرت درة
إلى كثرة الغاوين ركثا موسد 
 بخاتواه ووراد امتداحا ومنزلا
كما شقت الجيشان والعمر أجداثا غيركم في الدراقظ
بأنفسهم يوم التئيم وعرفك
ولكن حذار الصدريفن الذك قد أبقته العجا
قذاوا قرينا في نسيم ثرائره
وما العرف في دون المصاب فإنها
لما كان هذا ا إلى وحيية
تولى الكرات البيد غدرا بنفسه
وخير محمد وافد الاسمية 
 عيون وأحزام الإله سهام
وأع زينص الخبر هجرة واسع
ونشران حتى يدبر العذل القط
بسورته اذذلي بذكر وراءه
على المجد جواب وغبت به
عذوذك والأكوان فيه عداوة
وللفاضل النعمان غبض مخاصن
وتطذو على طول الجياد المسجود
حوائلها شيخت إذا نشوت رحيا
وقد تركت قفرا إليك وإنما
تقل على أجواده وعظام
حميت به بورة بسلي والذهيذ ببده
وأنت رحيته قلت للسحل أسوك
فرند هنوا صدقا أخو كل مكة
وعظم من عدر ظبي ورحمة
يحاطر ما أخلا أغيب وكمت
فيوما على الفضل القبيل يدا عن مرأته
 فكم قد يدر الخير في أحج اسطه
وليس لهم من حورة الآل همه
تطوذن أو لا أستطيع نظهره
وسوراء ما يتبعن بالناس كيف
فما روعة إلا كمدح المساعد

In [ ]:
model.save('keras.h5')